In [151]:
import torch
import torch_pruning as tp
from yolox.models import create_yolox_model

model = create_yolox_model(num_classes=20,name = 'yolox_custom',  
exp_path = '/workspace/tensorrt/YOLOX/exps/default/custom/test2/yolox_voc_tiny.py',
ckpt_path = '/workspace/tensorrt/YOLOX/0.2/best_ckpt.pth'
).cuda()
# 1. setup strategy (L1 Norm)
strategy = tp.strategy.L1Strategy() # or tp.strategy.RandomStrategy()

In [152]:

# 2. build dependency graph for resnet18
DG = tp.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1,3,224,224).cuda())



In [153]:

temp = []
temp2 = list(model.state_dict().keys())
num_list = [0,1,2,3]
#print(temp2)
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])

print(temp)
# backbone 만 프루닝


['backbone.backbone.stem.conv.conv', 'backbone.backbone.dark2[0].conv', 'backbone.backbone.dark2[1].conv1.conv', 'backbone.backbone.dark2[1].conv2.conv', 'backbone.backbone.dark2[1].conv3.conv', 'backbone.backbone.dark2[1].m[0].conv1.conv', 'backbone.backbone.dark2[1].m[0].conv2.conv', 'backbone.backbone.dark3[0].conv', 'backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[0].conv2.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv2.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv2.conv', 'backbone.backbone.dark4[0].conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[0].conv2.conv', 'backbone.backbone.dark4[1].m

In [154]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [155]:
from pruning_imagenet import Mask

In [156]:
m = Mask(model)

In [157]:
m.init_length()

In [158]:
m.init_mask(1, 0.272)
print(m.mask_index)

filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done


In [159]:
m.do_mask()
m.do_similar_mask()

mask Done
mask similar Done


In [160]:
m.if_zero()
print(m.mask_index)
print(m.similar_matrix)

layer: 24, number of nonzero weight is 2688, zero is 960
layer: 27, number of nonzero weight is 2688, zero is 960
layer: 30, number of nonzero weight is 4256, zero is 1520
layer: 33, number of nonzero weight is 1064, zero is 380
layer: 39, number of nonzero weight is 1064, zero is 380
layer: 45, number of nonzero weight is 1064, zero is 380
layer: 54, number of nonzero weight is 10752, zero is 3840
layer: 57, number of nonzero weight is 10752, zero is 3840
layer: 60, number of nonzero weight is 16872, zero is 6232
layer: 63, number of nonzero weight is 4256, zero is 1520
layer: 69, number of nonzero weight is 4256, zero is 1520
layer: 75, number of nonzero weight is 4256, zero is 1520
layer: 90, number of nonzero weight is 42624, zero is 15744
layer: 93, number of nonzero weight is 42624, zero is 15744
layer: 96, number of nonzero weight is 67488, zero is 24928
layer: 99, number of nonzero weight is 16872, zero is 6232
layer: 102, number of nonzero weight is 151848, zero is 56088
layer

In [161]:
model = m.model


In [162]:
dict ={}


for key, value in model.state_dict().items():
    temp = []
    if len(value.shape) ==4:
        print('pruning channel and layer name : ',len(value),',', key)
        for i in range(len(value)):
            temp
            if value[i].sum() == 0: 
                print('pruning channex index : ', i)
                #temp = torch.norm(value.view(len(value), -1) , p= 1, dim=1)
                #print(temp)
                #dict[key].append(i)
                temp.append(i)
    if temp:   
        dict[key]  = temp

pruning channel and layer name :  24 , backbone.backbone.stem.conv.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.0.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv2.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.1.conv3.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv2.conv.weight
pruning channel and layer name :  96 , backbone.backbone.dark3.0.conv.weight
pruning channel and layer name :  38 , backbone.backbone.dark3.1.conv1.conv.weight
pruning channex index :  5
pruning channex index :  11
pruning channex index :  12
pruning channex index :  13
pruning channex index :  14
pruning channex index :  23
pruning channex index :  25
pruning channex index :  26
pruning channex index :  32
pruning ch

In [163]:
print(dict)
#del dict['backbone.backbone.dark3.1.conv3.conv.weight'][0]

{'backbone.backbone.dark3.1.conv1.conv.weight': [5, 11, 12, 13, 14, 23, 25, 26, 32, 33], 'backbone.backbone.dark3.1.conv2.conv.weight': [0, 1, 3, 14, 16, 23, 26, 28, 35, 36], 'backbone.backbone.dark3.1.conv3.conv.weight': [2, 3, 6, 10, 18, 22, 25, 26, 33, 35, 37, 40, 43, 54, 61, 65, 72, 73, 74, 75], 'backbone.backbone.dark3.1.m.0.conv1.conv.weight': [0, 5, 6, 7, 10, 19, 21, 22, 29, 34], 'backbone.backbone.dark3.1.m.1.conv1.conv.weight': [3, 4, 11, 15, 18, 20, 22, 28, 32, 34], 'backbone.backbone.dark3.1.m.2.conv1.conv.weight': [4, 8, 10, 12, 14, 16, 28, 30, 32, 36], 'backbone.backbone.dark4.1.conv1.conv.weight': [9, 11, 13, 17, 19, 28, 29, 33, 34, 35, 39, 42, 50, 54, 55, 57, 60, 68, 71, 75], 'backbone.backbone.dark4.1.conv2.conv.weight': [1, 6, 11, 15, 18, 24, 26, 32, 35, 41, 45, 51, 53, 55, 59, 60, 62, 63, 67, 73], 'backbone.backbone.dark4.1.conv3.conv.weight': [1, 2, 6, 7, 14, 19, 21, 30, 32, 35, 38, 44, 45, 48, 49, 51, 53, 61, 62, 64, 69, 71, 76, 77, 79, 80, 101, 103, 109, 112, 113, 

In [164]:

temp = []
temp2 = list(dict.keys())
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])

print(temp)

['backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[1].conv1.conv', 'backbone.backbone.dark4[1].m[2].conv1.conv', 'backbone.backbone.dark5[2].conv1.conv', 'backbone.backbone.dark5[2].conv2.conv', 'backbone.backbone.dark5[2].conv3.conv', 'backbone.backbone.dark5[2].m[0].conv1.conv', 'backbone.backbone.dark5[2].m[0].conv2.conv', 'backbone.lateral_conv0.conv', 'backbone.C3_p4.conv1.conv', 'backbone.C3_p4.conv2.conv', 'backbone.C3_p4.conv3.conv', 'backbone.C3_p4.m[0].conv1.conv', 'backbone.C3_p4.m[0].conv2.conv', 'backbone.reduce_conv1.conv', 'backbone.bu_conv2.conv', 'backbone.bu_conv1.c

In [165]:
value = list(dict.values())
value

[[5, 11, 12, 13, 14, 23, 25, 26, 32, 33],
 [0, 1, 3, 14, 16, 23, 26, 28, 35, 36],
 [2, 3, 6, 10, 18, 22, 25, 26, 33, 35, 37, 40, 43, 54, 61, 65, 72, 73, 74, 75],
 [0, 5, 6, 7, 10, 19, 21, 22, 29, 34],
 [3, 4, 11, 15, 18, 20, 22, 28, 32, 34],
 [4, 8, 10, 12, 14, 16, 28, 30, 32, 36],
 [9,
  11,
  13,
  17,
  19,
  28,
  29,
  33,
  34,
  35,
  39,
  42,
  50,
  54,
  55,
  57,
  60,
  68,
  71,
  75],
 [1,
  6,
  11,
  15,
  18,
  24,
  26,
  32,
  35,
  41,
  45,
  51,
  53,
  55,
  59,
  60,
  62,
  63,
  67,
  73],
 [1,
  2,
  6,
  7,
  14,
  19,
  21,
  30,
  32,
  35,
  38,
  44,
  45,
  48,
  49,
  51,
  53,
  61,
  62,
  64,
  69,
  71,
  76,
  77,
  79,
  80,
  101,
  103,
  109,
  112,
  113,
  114,
  118,
  121,
  123,
  126,
  127,
  130,
  138,
  139,
  150],
 [2,
  8,
  12,
  19,
  20,
  27,
  28,
  29,
  31,
  33,
  39,
  46,
  47,
  50,
  58,
  64,
  69,
  70,
  72,
  75],
 [0, 3, 5, 7, 15, 17, 20, 26, 27, 32, 37, 40, 41, 42, 44, 45, 68, 69, 72, 74],
 [6,
  7,
  17,
  19,


In [166]:
for i in range(len(temp)):
    pruning_plan = DG.get_pruning_plan(eval(f"model.{temp[i]}"), tp.prune_conv_out_channel, value[i])
    print(pruning_plan)
    if DG.check_pruning_plan(pruning_plan):
        pruning_plan.exec()     


Conv2d(96, 38, kernel_size=(1, 1), stride=(1, 1), bias=False)

--------------------------------
          Pruning Plan
--------------------------------
User pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 38, kernel_size=(1, 1), stride=(1, 1), bias=False)) => ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 38, kernel_size=(1, 1), stride=(1, 1), bias=False)), Index=[5, 11, 12, 13, 14, 23, 25, 26, 32, 33], metric={'#params': 960}]

Coupled pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 38, kernel_size=(1, 1), stride=(1, 1), bias=False)) => BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(38, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)), Index=[5, 11, 12, 13, 14, 23, 25, 26, 32, 33], metric={'#params': 20}]
[ [DEP] BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(38, eps=0.001, momentum=0.03, affine=True, track_running_stats

In [167]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [168]:
ckpt_state = {
                "model": model.state_dict(),
            }

torch.save(ckpt_state, '0.4.pth')


